In [4]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("open_api_key")

## 청킹전략

### 부모-자식 분할

In [6]:
from langchain_community.document_loaders import TextLoader
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loaders = [TextLoader(r"C:\Users\kimji\Desktop\ProgramFile\Study\How_to_invest_money.txt", encoding="utf-8-sig")]
docs = []
for loader in loaders:
    docs.extend(loader.load())

parent_splitter  = RecursiveCharacterTextSplitter(chunk_size=1000)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

vectorstore = Chroma(
    collection_name="split_parents", embedding_function=OpenAIEmbeddings(api_key=api_key)
)

store = InMemoryStore()

retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter = parent_splitter,
)


retriever.add_documents(docs)

print(f"부모 문서 수 : {len(list(store.yield_keys()))}")




부모 문서 수 : 219


In [7]:
query = "what are the tpyes of investments?"

retrieved_docs = retriever.get_relevant_documents(query)
print(f"부모 문서 : {retrieved_docs[0].page_content}")

C:\Users\kimji\AppData\Local\Temp\ipykernel_37428\3443923552.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)


부모 문서 : There are five chief points to be considered in the selection of all
forms of investment. These are: (1) safety of principal and interest;
(2) rate of income; (3) convertibility into cash; (4) prospect of
appreciation in intrinsic value; (5) stability of market price.

Keeping these five general factors in mind, the present chapter will
discuss real-estate mortgages as a form of investment, both as adapted
to the requirements of private funds and of a business surplus.


In [8]:
sub_docs = vectorstore.similarity_search(query)
print(f"자식 문서 : {sub_docs[0].page_content}")

자식 문서 : forms of investment. These are: (1) safety of principal and interest;
(2) rate of income; (3) convertibility into cash; (4) prospect of
appreciation in intrinsic value; (5) stability of market price.


## 질의 변형

### 다중 질의 생성

In [5]:
# 다중 질의 생성 과정 모니터링을 위한 로깅 설정.
# 생성된 질의들 콘솔에서 확인
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [7]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import MultiQueryRetriever
from langchain_openai import ChatOpenAI

url = r"C:\Users\kimji\Desktop\ProgramFile\Study\How_to_invest_money.txt"
loaders  = [TextLoader(url, encoding="utf-8-sig")]

docs =[]
for loader in loaders:
    docs.extend(loader.load())

recursive_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
split_docs = recursive_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings(api_key=api_key)

vectorstore = Chroma.from_documents(documents=split_docs, embedding=embeddings)

llm = ChatOpenAI(model = "gpt-4o", temperature=0.2, api_key=api_key)

retriever = MultiQueryRetriever.from_llm(
    retriever= vectorstore.as_retriever(),
    llm = llm
)

question = "주식 투자를 처음 시작하려면 어떻게 해야 하나요?"

unique_docs = retriever.invoke(question)
print(f"\n결과 : {len(unique_docs)}개의 문서가 검색됨.")



INFO:langchain.retrievers.multi_query:Generated queries: ['주식 투자 초보자가 알아야 할 기본 단계는 무엇인가요?  ', '주식 투자를 처음 시작할 때 필요한 정보와 조언은 무엇인가요?  ', '주식 투자 입문자를 위한 가이드라인이나 팁이 있나요?']



결과 : 3개의 문서가 검색됨.


In [10]:
print(loaders)

In [9]:
print(docs)

[Document(metadata={'source': 'C:\\Users\\kimji\\Desktop\\ProgramFile\\Study\\How_to_invest_money.txt'}, page_content='The Project Gutenberg eBook of How to Invest Money\n    \nThis ebook is for the use of anyone anywhere in the United States and\nmost other parts of the world at no cost and with almost no restrictions\nwhatsoever. You may copy it, give it away or re-use it under the terms\nof the Project Gutenberg License included with this ebook or online\nat www.gutenberg.org. If you are not located in the United States,\nyou will have to check the laws of the country where you are located\nbefore using this eBook.\n\nTitle: How to Invest Money\n\nAuthor: George Garr Henry\n\nRelease date: November 28, 2010 [eBook #34463]\n                Most recently updated: January 7, 2021\n\nLanguage: English\n\nCredits: Produced by Julia Neufeld and the Online Distributed\n        Proofreading Team at https://www.pgdp.net (This file was\n        produced from images generously made available b

In [8]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = retriever,
    return_source_documents = True,
)

result = qa_chain.invoke({"query" :question})

print(f"답변 : {result["result"]}")
print("\n사용된 문서 : ")
for doc in result["source_documents"]:
    print(doc.page_content)

INFO:langchain.retrievers.multi_query:Generated queries: ['주식 투자 초보자가 알아야 할 기본 단계는 무엇인가요?  ', '주식 투자를 시작하기 위한 필수 준비 사항은 무엇인가요?  ', '처음 주식 투자에 입문할 때 필요한 조언이나 가이드는 무엇인가요?']


답변 : 주식 투자를 처음 시작하려면 다음과 같은 단계를 고려해 볼 수 있습니다:

1. **목표 설정**: 투자 목표를 명확히 설정하세요. 예를 들어, 장기적인 자산 증식을 목표로 할 것인지, 단기적인 수익을 추구할 것인지 결정해야 합니다.

2. **기본 지식 습득**: 주식 시장의 기본 원리와 작동 방식을 이해하세요. 주식의 가치 평가, 시장 동향, 경제 지표 등이 어떻게 주식 가격에 영향을 미치는지 공부하는 것이 중요합니다.

3. **투자 전략 수립**: 자신의 투자 스타일에 맞는 전략을 수립하세요. 예를 들어, 가치 투자, 성장 투자, 배당 투자 등 다양한 전략이 있습니다.

4. **증권 계좌 개설**: 주식을 매매하기 위해서는 증권 계좌가 필요합니다. 신뢰할 수 있는 증권사를 선택하고 계좌를 개설하세요.

5. **시장 조사 및 분석**: 관심 있는 주식이나 산업에 대해 철저히 조사하고 분석하세요. 기업의 재무 상태, 경쟁력, 시장 위치 등을 평가하는 것이 중요합니다.

6. **소액 투자 시작**: 처음에는 소액으로 시작하여 경험을 쌓는 것이 좋습니다. 시장의 변동성을 경험하면서 투자 감각을 키워나가세요.

7. **리스크 관리**: 투자에는 항상 리스크가 따릅니다. 포트폴리오를 다양화하고, 손실을 최소화할 수 있는 전략을 마련하세요.

8. **지속적인 학습**: 시장은 끊임없이 변화하므로, 지속적으로 학습하고 시장 동향을 파악하는 것이 중요합니다.

이러한 단계를 통해 주식 투자에 대한 이해를 높이고, 보다 효과적인 투자 결정을 내릴 수 있을 것입니다.

사용된 문서 : 
G. G. H.




HOW TO INVEST MONEY




I

GENERAL PRINCIPLES OF INVESTMENT
After learning how to judge the value of every form of investment, a man
may still be unsuccessful in the investment of money unless he 

### 가상 문서 임베딩 

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import MultiQueryRetriever
from langchain_openai import ChatOpenAI

url = r"C:\Users\kimji\Desktop\ProgramFile\Study\How_to_invest_money.txt"
loaders  = [TextLoader(url, encoding="utf-8-sig")]

docs =[]
for loader in loaders:
    docs.extend(loader.load())

recursive_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
split_docs = recursive_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings(api_key=api_key)

vectorstore = Chroma.from_documents(documents=split_docs, embedding=embeddings)

retriever= vectorstore.as_retriever()


In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_text_splitters import CharacterTextSplitter

def create_virtual_doc_chain():
    system = "당신은 고도로 숙련된 AI입니다."
    user = """
        주어진 질문 '{query}'에 대해 직접적으로 답변하는 가상의 문서를 생성하세요.
        문서의 크기는 {chunk_size} 글자 언저리여야 합니다.
    """

    prompts = ChatPromptTemplate([
        ("system",system),
        ("human", user)
        ]
    )

    llm = ChatOpenAI(model="gpt-4o", temperature=0.2, api_key=api_key)
    return prompts|llm|StrOutputParser()


def create_retrueval_chain():
    return RunnableLambda(lambda x : retriever.get_relevant_documents(x['virtual_doc']))

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def create_final_response_chain():
    final_prompts = ChatPromptTemplate.from_template(
        """ 다음 정보와 질문을 바탕으로 답변해주세요 :
        컨텍스트 : {context}
        질문 : {question}
        답변 : 
        """
    )
    final_llm = ChatOpenAI(model="gpt-4o", temperature=0.2, api_key=api_key)
    return final_prompts|final_llm


def print_input_output(input_data, output_data, step_name):
    print(f"\n---{step_name}---")
    print(f"Input : {input_data}")
    print(f"Output : {output_data}")
    print("-"*50)

def create_pipeline_with_loggin():
    virtual_doc_chain = create_virtual_doc_chain()
    retrieval_chain = create_retrueval_chain()
    final_response_chain = create_final_response_chain()

    def virtual_doc_step(x):
        result = {"virtual_doc" : virtual_doc_chain.invoke({
            "query" : x["question"],
            "chunk_size" : 200,
        })}

        print_input_output(x, result, "Virtual Doc Generation")
        return {**x, **result}
    
    def retrieval_step(x):
        result = {"retrieved_docs" : retrieval_chain.invoke(x)}
        print_input_output(x,result,"Document Retrieval")
        return {**x, **result}
    
    def context_formatting_step(x):
        result = {"context" : format_docs(x["retrieved_docs"])}
        print_input_output(x,result,"Context Formatting")
        return {**x, **result}
    
    def final_response_step(x):
        result = final_response_chain.invoke(x)
        print_input_output(x,result,"Final Respnose Generation")
        return result
    
    pipeline = (
        RunnableLambda(virtual_doc_step)
        | RunnableLambda(retrieval_step)
        | RunnableLambda(context_formatting_step)
        | RunnableLambda(final_response_step)
    )

    return pipeline

pipeline = create_pipeline_with_loggin()

question = "주식 시장의 변동성이 높을 때 투자 전략은 무엇인가요"
response = pipeline.invoke({"question" : question})
print(f"최종 답변 : {response.content}")



---Virtual Doc Generation---
Input : {'question': '주식 시장의 변동성이 높을 때 투자 전략은 무엇인가요'}
Output : {'virtual_doc': '주식 시장의 변동성이 높을 때는 신중한 투자 전략이 필요합니다. 첫째, 포트폴리오 다각화를 통해 리스크를 분산시키세요. 둘째, 방어주와 같은 안정적인 주식을 고려하세요. 셋째, 현금 비중을 늘려 기회를 대비하세요. 마지막으로, 장기적인 관점에서 시장의 일시적 변동에 흔들리지 않는 것이 중요합니다. 이러한 전략은 변동성 속에서도 안정적인 수익을 추구하는 데 도움이 될 수 있습니다.'}
--------------------------------------------------


INFO:langchain.retrievers.multi_query:Generated queries: ['주식 시장의 변동성이 클 때 리스크를 줄이기 위한 투자 전략은 무엇인가요?  ', '변동성이 높은 시장 상황에서 안정적인 수익을 추구하기 위한 포트폴리오 관리 방법은 어떤 것들이 있나요?  ', '주식 시장의 일시적 변동에 대응하기 위한 장기적인 투자 전략에는 어떤 것들이 포함될 수 있나요?']



---Document Retrieval---
Input : {'question': '주식 시장의 변동성이 높을 때 투자 전략은 무엇인가요', 'virtual_doc': '주식 시장의 변동성이 높을 때는 신중한 투자 전략이 필요합니다. 첫째, 포트폴리오 다각화를 통해 리스크를 분산시키세요. 둘째, 방어주와 같은 안정적인 주식을 고려하세요. 셋째, 현금 비중을 늘려 기회를 대비하세요. 마지막으로, 장기적인 관점에서 시장의 일시적 변동에 흔들리지 않는 것이 중요합니다. 이러한 전략은 변동성 속에서도 안정적인 수익을 추구하는 데 도움이 될 수 있습니다.'}
Output : {'retrieved_docs': [Document(metadata={'source': 'C:\\Users\\kimji\\Desktop\\ProgramFile\\Study\\How_to_invest_money.txt'}, page_content='which point to the beginning of a pronounced upward swing in securities,\nand if he can equally recognize the signs which indicate that the\nmovement has culminated, he can liquidate the securities which he bought\nat the inception of the rise or transfer them to some short-term issues\nwhose near approach to maturity will render them stable in price,\nallowing the downward swing to proceed without disturbing him. It is not\nexpected, of course, that the average business man will be able to\nrealize completely this ideal of investment,

## 검색 알고리즘